### Plan:
- clean up this space
- configure consts
    - move all consts to separate file
- make setup with loading all helpers etc
    - consider making  setup with accounts: `from helpers.interact import *`
    - setup CLI functions and fill with default values where possible?
    - need some useful toolset, one line for each command: deploy, deposit, withdraw
- create separate notebook for each of scenarios
- test different communications with rollup and ERC20 proxy, try to replace ERC20 proxy logic with some other contracts to check:
    - withdrawal precompile reentrancy
    - different kind of Ticket management
- calculate cost of withdrawal outbox overfill, check how this would work
    - make scenarios with different attempts to overfill outbox?
    - overfill with xtz withdrawals
    - overfill with fa withdrawals
    - overfill with callbacks from ERC20
- special scenario for checking costs of direct ticket deposits
- there are a lot more tests scenarios in the `tests.md` (check obsidian)
- `make_address_bytes` -> `make_address_bytes_hex`?

### Setup:

In [1]:
# TODO: move to consts, make it convenient to use

SMART_ROLLUP_ADDRESS = 'sr1G9N4WzRF79GeGKVd1vUDUg2NSrFGNGAfA'
XTZ_TICKET_HELPER = 'KT1MJxf4KVN3sosR99VRG7WBbWTJtAyWUJt9'
TEZOS_PRIVATE_KEY = 'edsk4XG4QyAj19dr78NNGH6dpXBtTnkmMdAkM9w5tUTCHaUP1pJaD5'
TEZOS_RPC_URL = 'https://rpc.tzkt.io/parisnet/'
ETHERLINK_RPC_URL = 'https://etherlink.dipdup.net'
ETHERLINK_PRIVATE_KEY = 'f463e320ed1bd1cd833e29efc383878f34abe6b596e5d163f51bb8581de6f8b8'
KERNEL_ADDRESS = '0x0000000000000000000000000000000000000000'
FA_WITHDRAWAL_PRECOMPILE = '0xff00000000000000000000000000000000000002'
XTZ_WITHDRAWAL_PRECOMPILE = '0xff00000000000000000000000000000000000001'
ETHERLINK_ROLLUP_NODE_URL = 'https://etherlink-rollup-paris.dipdup.net'
PRINT_DEBUG_LOG = False


if PRINT_DEBUG_LOG:
    import logging
    logging.basicConfig(level=logging.DEBUG)

### Changing variables:

In [ ]:
from getpass import getpass

ETHERLINK_RPC_URL = 'https://node.ghostnet.etherlink.com'
ETHERLINK_PRIVATE_KEY = getpass()

In [2]:
from scripts.helpers.utility import (
    get_tezos_client,
    make_address_bytes,
    get_etherlink_account,
    get_etherlink_web3,
    accent,
    echo_variable,
    wrap,
)
from scripts.helpers.contracts import (
    TokenBridgeHelper,
    TokenHelper,
    Ticketer,
)
import click
from scripts.helpers.addressable import (
    get_address,
    Addressable,
)

# TODO: provide consts as one object
# TODO: move setup to separate module in `helpers`
# TODO: make some dataclass with accounts
def setup():
    web3 = get_etherlink_web3(ETHERLINK_RPC_URL)
    etherlink_account = get_etherlink_account(web3, ETHERLINK_PRIVATE_KEY)
    tezos_account = get_tezos_client(TEZOS_RPC_URL, TEZOS_PRIVATE_KEY)

    wa_tezos_address = wrap(accent(tezos_account.key.public_key_hash()))
    wa_tezos_balance = wrap(accent(tezos_account.balance()))
    wa_etherlink_address = wrap(accent(etherlink_account.address))
    balance_wei = web3.eth.get_balance(etherlink_account.address)
    wa_etherlink_balance = wrap(accent(web3.from_wei(balance_wei, "ether")))

    click.echo('Setup:')
    click.echo('- Tezos account: ' + wa_tezos_address + ', balance: ' + wa_tezos_balance)
    click.echo('- Etherlink account: ' + wa_etherlink_address + ', balance: ' + wa_etherlink_balance)

    return web3, etherlink_account, tezos_account

# TODO: consider moving this to helpers:
def make_routing_info(router: Addressable, receiver: Addressable) -> bytes:
    # TODO: consider changing `make_address_bytes` so it really return bytes?
    ticketer_address_bytes = bytes.fromhex(make_address_bytes(get_address(router)))
    receiver = bytes.fromhex(make_address_bytes(get_address(receiver)))
    routing_info = receiver + ticketer_address_bytes
    return routing_info


web3, etherlink_account, tezos_account = setup()

Setup:
- Tezos account: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`, balance: `100.347038`
- Etherlink account: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`, balance: `0.869854055`


### Fund account:

In [4]:
from scripts.tezos import xtz_deposit
xtz_deposit.callback(
    xtz_ticket_helper=XTZ_TICKET_HELPER,
    tezos_private_key=TEZOS_PRIVATE_KEY,
    tezos_rpc_url=TEZOS_RPC_URL,
    amount=1_000_000,
    receiver_address=etherlink_account.address,
    smart_rollup_address=SMART_ROLLUP_ADDRESS,
)

Making XTZ deposit using Helper `KT1MJxf4KVN3sosR99VRG7WBbWTJtAyWUJt9`:
  - Executor: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Tezos RPC node: `https://rpc.tzkt.io/parisnet/`
  - XTZ deposit params:
      * Smart Rollup address: `sr1G9N4WzRF79GeGKVd1vUDUg2NSrFGNGAfA`
      * Receiver address: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
      * Amount: `1000000`
Successfully executed XTZ deposit, tx hash: `onpiUYKVCw5TkC1fnw7oYvH4TCweAubyVjDnHzLaecUCHqabEXS`


'onpiUYKVCw5TkC1fnw7oYvH4TCweAubyVjDnHzLaecUCHqabEXS'

### Deploy BulkWithdrawal:
- TODO: rename to BatchWithdrawal / organize helpers
- TODO: OR consider compiling contracts on the fly in the jupyter using web3?
- TODO: make wrapper that will print address and details of deploy?

In [4]:
from scripts.helpers.etherlink.contract import (
    EvmContractHelper,
    make_filename,
)

class BulkWithdrawalHelper(EvmContractHelper): ...


bulk_withdrawal = BulkWithdrawalHelper.originate_from_file(
    web3=web3,
    account=etherlink_account,
    filename=make_filename('BulkWithdrawal'),
    constructor_args=(
        FA_WITHDRAWAL_PRECOMPILE,
        XTZ_WITHDRAWAL_PRECOMPILE,
        450,
    )
)
bulk_withdrawal.address

'0x4A54796585667eb0fea583Baaec284E4F4358f0E'

### Deploy token

In [27]:
from scripts.tezos import deploy_token

token = deploy_token.callback(
    token_type='FA2',
    token_id=9000,
    total_supply=70_000_000,
    tezos_private_key=TEZOS_PRIVATE_KEY,
    tezos_rpc_url=TEZOS_RPC_URL,
    skip_confirm=True,
)
token.address

'KT1Kf9c5ufLWE89N4zBWuBqsEMa5G2gFDRSr'

### Deploy bridge:

In [7]:
from scripts.bridge_token import bridge_token

tezos_bridge_contracts = bridge_token.callback(
    token_address=token.address,
    token_type='FA2',
    token_id=9000,
    token_decimals=0,
    token_symbol='vSecT',
    token_name='Various Security Testing Scenarios',
    tezos_private_key=TEZOS_PRIVATE_KEY,
    tezos_rpc_url=TEZOS_RPC_URL,
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL,
    kernel_address=KERNEL_ADDRESS,
    skip_confirm=True
)

ticketer = tezos_bridge_contracts['ticketer']
# TODO: consider replacing erc20 here with helper
erc20_address = tezos_bridge_contracts['erc20']
token_bridge_helper = tezos_bridge_contracts['token_bridge_helper']

Deploying bridge contracts for `vSecT`:
  - Token contract: `KT1Kf9c5ufLWE89N4zBWuBqsEMa5G2gFDRSr`
  - Token id: `9000`

Deploying Ticketer for `vSecT`:
  - Deployer: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Tezos RPC node: `https://rpc.tzkt.io/parisnet/`
  - Params:
      * Token type: `FA2`
      * Token address: `KT1Kf9c5ufLWE89N4zBWuBqsEMa5G2gFDRSr`
      * Token id: `9000`
      * Token symbol: `vSecT`
      * Token name: `Various Security Testing Scenarios`
      * Token decimals: `0`
Successfully deployed Ticketer, address: `KT1TQiSYR6VhFHso5xkEYR3Fgg5Vsz8iRaK7`

Deploying ERC20 Proxy for `vSecT`:
  - Deployer: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://etherlink.dipdup.net`
  - Constructor params:
      * Ticketer address bytes: `0x01ce839f3e5187c0abec444760a58e950b675a334100`
      * Content bytes: `0x0707000005090a000000c00502000000ba07040100000010636f6e74726163745f616464726573730a000000244b54314b6639633575664c574538394e347a425775427173454

### Reload token / bridge:
- TODO: add special functions to simply save / load set of tokens bridges?
- the best would be one function that either deploys token+bridge / either loads it from some `test-scenario-setup.json`?

In [3]:
from scripts.helpers.contracts import TokenBridgeHelper, TokenHelper, Ticketer

token = TokenHelper.get_cls('FA2').from_address(tezos_account, 'KT1Kf9c5ufLWE89N4zBWuBqsEMa5G2gFDRSr')
token_bridge_helper = TokenBridgeHelper.from_address(tezos_account, 'KT1SSfdYB4JFTJoqVUe6yjVX9urLXiXge6e4')

# TODO: add method to get ERC20 proxy address? (or it would be better to get ERC20 helper)
erc20_address = token_bridge_helper.contract.storage()['erc_proxy'].hex()
erc20_address = web3.to_checksum_address(erc20_address)
ticketer = token_bridge_helper.get_ticketer()

In [4]:
from scripts.helpers.etherlink import (
    Erc20ProxyHelper,
    make_filename,
    load_contract_type,
)

Erc20ProxyContract = load_contract_type(web3, make_filename('ERC20Proxy'))
erc20 = Erc20ProxyHelper.from_address(Erc20ProxyContract, web3, etherlink_account, erc20_address)
erc20.address

'0x03E39FF2b379FBcd9284Ab457113D82fF4daBBF4'

### Deposit

In [5]:
from scripts import tezos

result = tezos.deposit.callback(
    token_bridge_helper_address=token_bridge_helper.address,
    amount=1_000,
    receiver_address=bulk_withdrawal.address,
    smart_rollup_address=SMART_ROLLUP_ADDRESS,
    tezos_private_key=TEZOS_PRIVATE_KEY,
    tezos_rpc_url=TEZOS_RPC_URL
)

Making deposit using Helper `KT1SSfdYB4JFTJoqVUe6yjVX9urLXiXge6e4`:
  - Executor: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Tezos RPC node: `https://rpc.tzkt.io/parisnet/`
  - Ticketer: `KT1TQiSYR6VhFHso5xkEYR3Fgg5Vsz8iRaK7`
  - Deposit params:
      * Smart Rollup address: `sr1G9N4WzRF79GeGKVd1vUDUg2NSrFGNGAfA`
      * Receiver address: `0x4A54796585667eb0fea583Baaec284E4F4358f0E`
      * Amount: `1000`
Successfully executed Deposit, tx hash: `opCsjq3f8tFC7i3ASbyHoS9vC7Nu9LvDaT67sprVKCNHxmzi9e4`


- TODO: replace this call withdraw with `withdaw` CLI maybe?

In [20]:
# TODO: make withdraw function too:
from scripts.etherlink import withdraw

withdraw.callback(
    erc20_proxy_address=erc20_address,
    tezos_side_router_address=ticketer.address,
    amount=1,
    # TODO: consider changing this to ticketer_address_bytes_hex ?
    ticketer_address_bytes=make_address_bytes(ticketer.address),
    # TODO: consider changing this to ticket_content_bytes_hex ?
    ticket_content_bytes=ticketer.read_content().to_bytes_hex(),
    receiver_address=get_address(tezos_account),
    withdraw_precompile=bulk_withdrawal.address,
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL,
)

Making FA withdrawal, ERC20 token: `0x03E39FF2b379FBcd9284Ab457113D82fF4daBBF4`:
  - Executor: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://etherlink.dipdup.net`
  - Withdrawal params:
      * Ticket owner: `0x03E39FF2b379FBcd9284Ab457113D82fF4daBBF4`
      * Receiver: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
      * Router: `KT1TQiSYR6VhFHso5xkEYR3Fgg5Vsz8iRaK7`
      * Routing info: `0x0000d1b03118754a8e193b3e5d3a7cded976f4fd425601ce839f3e5187c0abec444760a58e950b675a334100`
      * Amount: `1`
      * Ticketer address bytes: `0x01ce839f3e5187c0abec444760a58e950b675a334100`
      * Content bytes: `0x0707000005090a000000c00502000000ba07040100000010636f6e74726163745f616464726573730a000000244b54314b6639633575664c574538394e347a425775427173454d61354732674644525372070401000000046e616d650a00000022566172696f75732053656375726974792054657374696e67205363656e6172696f730704010000000673796d626f6c0a00000005765365635407040100000008746f6b656e5f69640a000000043930303007040

'0x25e74c176caa64f1cc202ed01e4351feb837ab5c007b868127279942479abfd7'

In [6]:
'''
def call_withdraw(contract, ticket_owner, routing_info, amount, ticketer, content):
    # withdraw(address,bytes,uint256,bytes22,bytes)
    transaction = contract.functions.withdraw(
        ticket_owner,
        routing_info,
        amount,
        ticketer,
        content
    ).build_transaction({
        'from': account.address,
        'nonce': w3.eth.get_transaction_count(account.address),
        'chainId': w3.eth.chain_id
    })

    signed_txn = w3.eth.account.sign_transaction(transaction, ETHERLINK_PRIVATE_KEY)
    txn_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)
    txn_receipt = w3.eth.wait_for_transaction_receipt(txn_hash)

    return txn_receipt

ticket_owner = erc20_address
ticketer_address_bytes = bytes.fromhex(make_address_bytes(ticketer.address))
routing_info = make_routing_info(ticketer, tezos_account)
amount = 1
content = bytes.fromhex(ticketer.read_content().to_bytes_hex())

receipt = call_withdraw(bulk_withdrawal.contract, ticket_owner, routing_info, amount, ticketer_address_bytes, content)
print(receipt)
'''

"\ndef call_withdraw(contract, ticket_owner, routing_info, amount, ticketer, content):\n    # withdraw(address,bytes,uint256,bytes22,bytes)\n    transaction = contract.functions.withdraw(\n        ticket_owner,\n        routing_info,\n        amount,\n        ticketer,\n        content\n    ).build_transaction({\n        'from': account.address,\n        'nonce': w3.eth.get_transaction_count(account.address),\n        'chainId': w3.eth.chain_id\n    })\n\n    signed_txn = w3.eth.account.sign_transaction(transaction, ETHERLINK_PRIVATE_KEY)\n    txn_hash = w3.eth.send_raw_transaction(signed_txn.rawTransaction)\n    txn_receipt = w3.eth.wait_for_transaction_receipt(txn_hash)\n\n    return txn_receipt\n\nticket_owner = erc20_address\nticketer_address_bytes = bytes.fromhex(make_address_bytes(ticketer.address))\nrouting_info = make_routing_info(ticketer, tezos_account)\namount = 1\ncontent = bytes.fromhex(ticketer.read_content().to_bytes_hex())\n\nreceipt = call_withdraw(bulk_withdrawal.cont

In [133]:
(
    bulk_withdrawal.address,
    ticket_owner,
    ticketer_address_bytes.hex(),
    receiver.hex(),
    routing_info.hex(),
    amount,
    content.hex(),
)

('0x4fe8C5E99F000b9151E52dbFAA0297B20a0338a0',
 '0x3dFF505A2A69e6e0b05fDB71b5F6DDd514fDaF47',
 '019befe90f2f9e2fa87f79f4cee695ab6d88040ac500',
 '0000d1b03118754a8e193b3e5d3a7cded976f4fd4256',
 '0000d1b03118754a8e193b3e5d3a7cded976f4fd4256019befe90f2f9e2fa87f79f4cee695ab6d88040ac500',
 2,
 '0707000005090a000000b10502000000ab07040100000010636f6e74726163745f616464726573730a000000244b54314845726275356f70544d6b7434503833695868454a783147317556566e5043316f070401000000046e616d650a00000013436f6e7472616374205769746864726177616c0704010000000673796d626f6c0a000000074357205445535407040100000008746f6b656e5f69640a0000000237370704010000000a746f6b656e5f747970650a00000003464132')

### Native deposit:

In [8]:
from scripts.tezos import xtz_deposit
xtz_deposit.callback(
    xtz_ticket_helper=XTZ_TICKET_HELPER,
    tezos_private_key=TEZOS_PRIVATE_KEY,
    tezos_rpc_url=TEZOS_RPC_URL,
    amount=1_000_000,
    # receiver_address=etherlink_account.address,
    receiver_address=bulk_withdrawal.address,
    smart_rollup_address=SMART_ROLLUP_ADDRESS,
)

Making XTZ deposit using Helper `KT1MJxf4KVN3sosR99VRG7WBbWTJtAyWUJt9`:
  - Executor: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Tezos RPC node: `https://rpc.tzkt.io/parisnet/`
  - XTZ deposit params:
      * Smart Rollup address: `sr1G9N4WzRF79GeGKVd1vUDUg2NSrFGNGAfA`
      * Receiver address: `0x4A54796585667eb0fea583Baaec284E4F4358f0E`
      * Amount: `1000000`
Successfully executed XTZ deposit, tx hash: `ooaDNp1gNjeHUypXXHv4uj3B3RMYXT3fiMYSJG71jrN3BPNuz8j`


'ooaDNp1gNjeHUypXXHv4uj3B3RMYXT3fiMYSJG71jrN3BPNuz8j'

In [10]:
bulk_withdrawal.address

'0x4A54796585667eb0fea583Baaec284E4F4358f0E'

### Native withdrawal

In [ ]:
from scripts.etherlink import xtz_withdraw

xtz_withdraw.callback(
    amount=10**12,
    # xtz_withdraw_precompile=XTZ_WITHDRAWAL_PRECOMPILE,
    xtz_withdraw_precompile=bulk_withdrawal.address,
    receiver_address='tz1burnburnburnburnburnburnburjAYjjX',
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL
)

### XTZ deposit to contract with `receive()` test
- TODO: consider compiling the `DepositTester` contract inplace using web3?
- TODO: move this to separate scenario.ipynb
- checking that Kernel call doesn't run `receive()` when EOA call runs it

In [5]:
from scripts.helpers.etherlink.contract import (
    EvmContractHelper,
    make_filename,
)

class DepositTesterHelper(EvmContractHelper): ...


deposit_helper = DepositTesterHelper.originate_from_file(
    web3=web3,
    account=etherlink_account,
    filename=make_filename('DepositTester'),
    constructor_args=(
        XTZ_WITHDRAWAL_PRECOMPILE,
        'tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc',
    )
)
deposit_helper.address

'0x0387876E770c0b7CB61E60D62c7BB2F3f009bdE4'

- TODO: make some helper for this deposit that loaded once in the ipynb and can be easily reused next

In [9]:
from scripts.tezos import xtz_deposit

result = xtz_deposit.callback(
    xtz_ticket_helper=XTZ_TICKET_HELPER,
    amount=1,
    receiver_address=deposit_helper.address,
    smart_rollup_address=SMART_ROLLUP_ADDRESS,
    tezos_private_key=TEZOS_PRIVATE_KEY,
    tezos_rpc_url=TEZOS_RPC_URL
)
result

Making XTZ deposit using Helper `KT1MJxf4KVN3sosR99VRG7WBbWTJtAyWUJt9`:
  - Executor: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Tezos RPC node: `https://rpc.tzkt.io/parisnet/`
  - XTZ deposit params:
      * Smart Rollup address: `sr1LmwJbvp8ZGwTRzAoy8mheGV5xX2kYef1L`
      * Receiver address: `0x0387876E770c0b7CB61E60D62c7BB2F3f009bdE4`
      * Amount: `1`
Successfully executed XTZ deposit, tx hash: `ooFyqDwUHQhTA5TYdEL2xjjgkXe5LabZxuWc46oS58DudiAKVbf`


'ooFyqDwUHQhTA5TYdEL2xjjgkXe5LabZxuWc46oS58DudiAKVbf'

- TODO: move to separate function `legacy_transfer` or smth similar

In [16]:
transaction = {
    'to': deposit_helper.address,
    'gas': 2000000,
    'gasPrice': web3.to_wei('1', 'gwei'),
    'value': web3.to_wei(1, 'microether'),
    'nonce': web3.eth.get_transaction_count(etherlink_account.address),
}

signed_txn = etherlink_account.sign_transaction(transaction)
txn_hash = web3.eth.send_raw_transaction(signed_txn.rawTransaction)
txn_hash

HexBytes('0xbf0338fe75667cf5d98855e7310ab710e20a2caa0e6f44bd007026ae3d01843f')

In [ ]:
# TODO: check that first transaction (called by Kernel during deposit) doesn't created any logs
# TODO: check that second transaction (called by etherlink_account) created logs

### Withdrawal precompile reentrancy
- TODO: move to separate scenario too

In [8]:
# TODO: this is prototype of the functions that can be used for all scenarios
# TODO: some of the code might be moved directly to scripts.helpers I suppose
from scripts.tezos import deposit
from scripts.helpers.etherlink.contract import (
    EvmContractHelper,
    make_filename,
)
from web3.types import TxReceipt
from scripts import tezos


class TokenProxyTesterHelper(EvmContractHelper):
    ...
    def set_parameters(
        self,
        # TODO: make some of the parameters optional and if not provided check the current storage value?
        withdrawal_precompile: str,
        routing_info: bytes,
        amount: int,
        ticketer: bytes,
        content: bytes,
        calls_count: int,
    ) -> TxReceipt:
        """Replaces TokenProxyHelper parameters with provided"""

        call = self.contract.functions.setParameters(
            withdrawal_precompile, routing_info, amount, ticketer, content, calls_count
        )

        # TODO: this build_transaction can be created inside self.legacy_send
        #       but for xtz withdrawal precompile call we need `value` to be provided too
        transaction = call.build_transaction(
            {
                'from': self.account.address,
                'nonce': self.web3.eth.get_transaction_count(self.account.address),
                'chainId': self.web3.eth.chain_id,
            }
        )

        return self.legacy_send(transaction)


def deploy_token_proxy_tester(ticket, amount, router, receiver, calls_count = 0):
    routing_info = make_routing_info(router, receiver)
    ticketer_address_bytes = bytes.fromhex(make_address_bytes(ticket.ticketer))

    # TODO: consider adding `to_bytes` method
    ticket_content_bytes = bytes.fromhex(ticket.content.to_bytes_hex())

    return TokenProxyTesterHelper.originate_from_file(
        web3=web3,
        account=etherlink_account,
        filename=make_filename('TokenProxyTester'),
        constructor_args=(
            FA_WITHDRAWAL_PRECOMPILE,
            routing_info,
            amount,
            ticketer_address_bytes,
            ticket_content_bytes,
            calls_count,
        )
    )

'''
def deposit(receiver, amount):
    return tezos.deposit.callback(
        token_bridge_helper_address=token_bridge_helper.address,
        amount=amount,
        receiver_address=receiver,
        smart_rollup_address=SMART_ROLLUP_ADDRESS,
        tezos_private_key=TEZOS_PRIVATE_KEY,
        tezos_rpc_url=TEZOS_RPC_URL
    )
'''

'\ndef deposit(receiver, amount):\n    return tezos.deposit.callback(\n        token_bridge_helper_address=token_bridge_helper.address,\n        amount=amount,\n        receiver_address=receiver,\n        smart_rollup_address=SMART_ROLLUP_ADDRESS,\n        tezos_private_key=TEZOS_PRIVATE_KEY,\n        tezos_rpc_url=TEZOS_RPC_URL\n    )\n'

### Reload TokenProxyTester
- This logic should be moved to the Contract.from_address or some similar helpers, this have to be simplified

In [7]:
from scripts.helpers.etherlink import (
    make_filename,
    load_contract_type,
)

TokenProxyTesterContract = load_contract_type(web3, make_filename('TokenProxyTester'))
token_proxy_tester = TokenProxyTesterHelper.from_address(TokenProxyTesterContract, web3, etherlink_account, token_proxy_tester.address)
token_proxy_tester.address

### Deploy TicketRouterTester:

In [30]:
from scripts.helpers.contracts import TicketRouterTester
import click

# if we decide to make a function:
silent = False
tezos_rpc_url = TEZOS_RPC_URL

# TODO: do I need some function which would make origination in the same way CLI works?
# TODO: consider change `originate` to `make_originate_opg`
#       and also it would be good to have real `originate` with wait and returning actual object
# TODO: provide setup tools for ticket-router-tester in CLI too? OR at least during deployment?

if not silent:
    click.echo('Deploying TicketRouterTester:')
    echo_variable('  - ', 'Deployer', tezos_account.key.public_key_hash())
    echo_variable('  - ', 'Tezos RPC node', tezos_rpc_url)

origination_opg = TicketRouterTester.originate(tezos_account).send()
tezos_account.wait(origination_opg)
ticket_router_tester = TicketRouterTester.from_opg(tezos_account, origination_opg)

if not silent:
    click.echo(
        'Successfully deployed TicketRouterTester, address: ' + wrap(accent(ticket_router_tester.address))
    )

Deploying TicketRouterTester:
  - Deployer: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Tezos RPC node: `https://rpc.tzkt.io/parisnet/`
Successfully deployed TicketRouterTester, address: `KT1RZKsU9CD4siruhkxwzuwshwmmdXeM2ia4`


### Reload TicketRouterTester

In [9]:
from scripts.helpers.contracts import TicketRouterTester
ticket_router_tester = TicketRouterTester.from_address(tezos_account, 'KT1RZKsU9CD4siruhkxwzuwshwmmdXeM2ia4')
ticket_router_tester.address

'KT1RZKsU9CD4siruhkxwzuwshwmmdXeM2ia4'

## Scenario A: `TokenProxyTester` withdraws tickets twice during withdrawal, but it has these tickets
- setup:
    - tester originated with callsCount=0, amount=10
    - deposit 1000 tickets to the proxy from L1 via kernel
    - tester resetup to have callsCount=1, amount=10
- test:
    - user calls `fa_withdrawal_precompile` to withdraw 1 ticket from the tester
        * tester withdraws 10 more tickets in the internal call
    - check transaction succeed
    - check that 989 tickets left on tester
    - check that 2 withdrawal events created, each for 2 tickets
    - check that 2 outbox messages created
    - deposit 21 tickets to the proxy from L1 via kernel
        * tester withdraws 10 more tickets in the internal call
    - check that 1 outbox message created
    - check that 1 deposit event created
    - check that 1000 tickets left on tester
    - user tries to withdraw > 990 tickets from tester and transaction fails
    - user tries tp withdraw 990 tickets and succeed
        * tester withdraws 10 more tickets in the internal call
    - check that two withdrawal events and 2 outbox messages created
    - deposit 15 tickets to the proxy from L1 via kernel
        * tester tries to withdraw 10 more tickets in the internal call but not succeed
    - check deposit succeed, deposit event created and tester recieves 15 tickets

In [37]:
withdraw_amount = 10
receiver = get_address(tezos_account)
ticket = ticketer.read_ticket()

token_proxy_tester = deploy_token_proxy_tester(ticket, withdraw_amount, ticketer, receiver)
token_proxy_tester.address

'0x11e1D330879bC3cc54c0F698Dd080F326C6BD5c4'

In [38]:
def etherlink_address_to_bytes(etherlink_address):
    return bytes.fromhex(etherlink_address.replace('0x', ''))

# TODO: do I need some L2Addressable type for these kind of logic?
# TODO: rename another make_routing_info to make_withdrawal_routing_info?
def make_deposit_routing_info(receiver, router):
    return (
        etherlink_address_to_bytes(receiver.address)
        + etherlink_address_to_bytes(router.address)
    )

In [39]:
# NOTE: this is wrong deposit, it uses Token Bridge Helper to deposit
# - need to either make another Token Bridge Helper for token_proxy_tester as ERC20
# - either deploy TicketRouterTester on L1 and reuse it for deposits
# deposit(token_proxy_tester.address, 10)

# TODO: make a function to setup TicketRouterTester to different modes? maybe even CLI?
target = SMART_ROLLUP_ADDRESS
deposit_info = make_deposit_routing_info(token_proxy_tester, token_proxy_tester)

click.echo(
    'Setting up ' + accent('TicketRouterTester')
    + ' to the ' + accent('rollupDeposit') + ' mode:'
)
echo_variable('  - ', 'Target', tezos_account.key.public_key_hash())
echo_variable('  - ', 'Routing Info', '0x' + deposit_info.hex())

set_deposit_opg = ticket_router_tester.set_rollup_deposit(
    target=target,
    routing_info=deposit_info,
).send()

tezos_account.wait(set_deposit_opg)
click.echo(
    'Successfully set, tx hash: ' + wrap(accent(set_deposit_opg.hash()))
)

Setting up TicketRouterTester to the rollupDeposit mode:
  - Target: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Routing Info: `0x11e1d330879bc3cc54c0f698dd080f326c6bd5c411e1d330879bc3cc54c0f698dd080f326c6bd5c4`
Successfully set, tx hash: `oos6U5GPc1q8w6evDe8fXkdzY6CDP448trkEt5hJp3mSeyqqJKY`


In [40]:
# TODO: create echo.py? and move all these echo helpers here?
# OR: maybe it would be OK to move this logic into tokens? not sure this is good idea

def format_token_info(token):
    token_dict = token.as_dict()
    if 'fa2' in token_dict:
        return (
            accent('FA2') + ' token, address: ' + wrap(accent(token.address))
            + ', id: ' + wrap(accent(token.token_id))
        )
    elif 'fa12' in token_dict:
        return (
            accent('FA1.2') + 'token, address' + wrap(accent(token.address))
        )
    else:
        return (
            'Undefined token type, address: ' + wrap(accent(token.address))
            + ', id: ' + wrap(accent(token.token_id))
        )

click.echo(format_token_info(ticketer.get_token()))

FA2 token, address: `KT1Kf9c5ufLWE89N4zBWuBqsEMa5G2gFDRSr`, id: `9000`


### TODO: add these steps to the setup?

In [41]:
# TODO: make a function to wrap tokens to tickets?

amount = 1_000_000
click.echo(
    'Wrapping ' + format_token_info(ticketer.get_token()) + ' tokens to tickets:'
)
echo_variable('  - ', 'Token holder', tezos_account.key.public_key_hash())
echo_variable('  - ', 'Ticketer', ticketer.address)
echo_variable('  - ', 'Amount', amount)

wrap_opg = tezos_account.bulk(
    ticketer.get_token().allow(tezos_account, ticketer),
    ticketer.deposit(amount),
).send()

tezos_account.wait(wrap_opg)
click.echo(
    'Successfully wrapped, tx hash: ' + wrap(accent(wrap_opg.hash()))
)

Wrapping FA2 token, address: `KT1Kf9c5ufLWE89N4zBWuBqsEMa5G2gFDRSr`, id: `9000` tokens to tickets:
  - Token holder: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Ticketer: `KT1TQiSYR6VhFHso5xkEYR3Fgg5Vsz8iRaK7`
  - Amount: `1000000`
Successfully wrapped, tx hash: `ooxU1sr54wkDVUYhs9vETch23DYirfyahNruuHPHNJ45sGsrqFm`


In [42]:
# TODO: add ticket transfer function?

def transfer_tickets(ticket, sender, receiver):
    click.echo('Transfering tickets:')
    echo_variable('  - ', 'Sender', get_address(sender))
    echo_variable('  - ', 'Ticketer address', ticketer.address)
    echo_variable('  - ', 'Ticket content', '0x' + ticketer.read_content().to_bytes_hex())
    echo_variable('  - ', 'Amount', ticket.amount)
    echo_variable('  - ', 'Receiver', get_address(receiver))

    transfer_opg = ticket.transfer(receiver).send()

    tezos_account.wait(transfer_opg)
    click.echo(
        'Successfully transfered, tx hash: ' + wrap(accent(transfer_opg.hash()))
    )
    return transfer_opg

ticket = ticketer.read_ticket(tezos_account)
ticket, _ = ticket.split(1_000)

opg = transfer_tickets(
    ticket=ticket,
    sender=tezos_account,
    receiver=ticket_router_tester,
)

Transfering tickets:
  - Sender: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Ticketer address: `KT1TQiSYR6VhFHso5xkEYR3Fgg5Vsz8iRaK7`
  - Ticket content: `0x0707000005090a000000c00502000000ba07040100000010636f6e74726163745f616464726573730a000000244b54314b6639633575664c574538394e347a425775427173454d61354732674644525372070401000000046e616d650a00000022566172696f75732053656375726974792054657374696e67205363656e6172696f730704010000000673796d626f6c0a00000005765365635407040100000008746f6b656e5f69640a00000004393030300704010000000a746f6b656e5f747970650a00000003464132`
  - Amount: `1000`
  - Receiver: `KT1RZKsU9CD4siruhkxwzuwshwmmdXeM2ia4`
Successfully transfered, tx hash: `ooQ9F4qaJriSpvPxANAyJjtAUdCKkkbEqy1yPsmHgRHUh8yphrQ`


### Getting ticket table value:

In [43]:
from urllib.parse import urlparse, urlunparse, urlencode
import requests
from typing import Any, TypedDict
from scripts.helpers.ticket import Ticket

def make_ticket_table_key(ticket: Ticket, owner_address: str):
    kernel_address = '0000000000000000000000000000000000000000'
    address = owner_address.replace('0x', '').lower()
    # TODO: is it required to add leading zeroes?
    ticket_hash_hex = hex(ticket.hash())[2:].lower()
    key = f'/evm/world_state/eth_accounts/{kernel_address}/ticket_table/{ticket_hash_hex}/{address}'
    return key


def get_durable_storage_value(rollup_node_url: str, key: str):
    parts = urlparse(rollup_node_url)
    parts = parts._replace(
        path=f'global/block/head/durable/wasm_2_0_0/value',
        query=urlencode(dict(key=key)),
    )
    url = urlunparse(parts)
    return requests.get(url).json()

def get_tickets_count(rollup_node_url: str, ticket: Ticket, owner_address: str):
    storage_value = get_durable_storage_value(
        rollup_node_url,
        make_ticket_table_key(ticket, owner_address),
    )
    if storage_value is None:
        raise Exception('No tickets')  # TODO: need to find a way how to handle this, maybe return 0?
    return int.from_bytes(bytes.fromhex(storage_value), 'little')

result = get_tickets_count(ETHERLINK_ROLLUP_NODE_URL, ticket, token_proxy_tester.address)
result

1000

### Setup to make 1 call:
- need to find a way to simplify this setup, maybe reuse some logic from deployment, as this reuse the same parameters:

In [44]:
# TODO: make a function to setup TokenProxyTester with echo?

withdraw_amount = 10
receiver = tezos_account
ticket = ticketer.read_ticket()
ticketer_address_bytes = bytes.fromhex(make_address_bytes(ticket.ticketer))
ticket_content_bytes = bytes.fromhex(ticket.content.to_bytes_hex())

result = token_proxy_tester.set_parameters(
    withdrawal_precompile=FA_WITHDRAWAL_PRECOMPILE,
    routing_info=make_routing_info(ticketer, receiver),
    amount=withdraw_amount,
    ticketer=ticketer_address_bytes,
    content=ticket_content_bytes,
    calls_count=1
)

result

AttributeDict({'transactionHash': HexBytes('0x400ec349fd524855062c0d7dac9fe2f76a11c13dd5e3de6a072e04de149ad7d5'),
 'transactionIndex': 0,
 'blockHash': HexBytes('0x28b8f38ad35df8b6987a67fcd5dd63a69406285b5650394c79cedaffdb110c78'),
 'blockNumber': 37732,
 'from': '0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677',
 'to': '0x11e1D330879bC3cc54c0F698Dd080F326C6BD5c4',
 'cumulativeGasUsed': 2843069,
 'effectiveGasPrice': 1000000000,
 'gasUsed': 2843069,
 'logs': [],
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000

In [45]:
# TODO: make withdraw function too:
from scripts.etherlink import withdraw

withdraw.callback(
    erc20_proxy_address=token_proxy_tester.address,
    tezos_side_router_address=ticketer.address,
    amount=1,
    # TODO: consider changing this to ticketer_address_bytes_hex ?
    ticketer_address_bytes=make_address_bytes(ticketer.address),
    # TODO: consider changing this to ticket_content_bytes_hex ?
    ticket_content_bytes=ticketer.read_content().to_bytes_hex(),
    receiver_address=get_address(tezos_account),
    withdraw_precompile=FA_WITHDRAWAL_PRECOMPILE,
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL,
)

Making FA withdrawal, ERC20 token: `0x11e1D330879bC3cc54c0F698Dd080F326C6BD5c4`:
  - Executor: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://etherlink.dipdup.net`
  - Withdrawal params:
      * Ticket owner: `0x11e1D330879bC3cc54c0F698Dd080F326C6BD5c4`
      * Receiver: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
      * Router: `KT1TQiSYR6VhFHso5xkEYR3Fgg5Vsz8iRaK7`
      * Routing info: `0x0000d1b03118754a8e193b3e5d3a7cded976f4fd425601ce839f3e5187c0abec444760a58e950b675a334100`
      * Amount: `1`
      * Ticketer address bytes: `0x01ce839f3e5187c0abec444760a58e950b675a334100`
      * Content bytes: `0x0707000005090a000000c00502000000ba07040100000010636f6e74726163745f616464726573730a000000244b54314b6639633575664c574538394e347a425775427173454d61354732674644525372070401000000046e616d650a00000022566172696f75732053656375726974792054657374696e67205363656e6172696f730704010000000673796d626f6c0a00000005765365635407040100000008746f6b656e5f69640a000000043930303007040

'0x277429bd430e06cefe7425ce11dd6080c4f4632abdaed0c5c8ba61ea7867cab2'

- checking that it has 11 tokens less (1_000 - 11 = 989)

### Need to wait until rollup processes transaction, not sure what is the best way to do this

In [46]:
result = get_tickets_count(ETHERLINK_ROLLUP_NODE_URL, ticket, token_proxy_tester.address)
result

989

In [47]:
ticket = ticketer.read_ticket(tezos_account)
ticket, _ = ticket.split(21)

opg = transfer_tickets(
    ticket=ticket,
    sender=tezos_account,
    receiver=ticket_router_tester,
)

Transfering tickets:
  - Sender: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Ticketer address: `KT1TQiSYR6VhFHso5xkEYR3Fgg5Vsz8iRaK7`
  - Ticket content: `0x0707000005090a000000c00502000000ba07040100000010636f6e74726163745f616464726573730a000000244b54314b6639633575664c574538394e347a425775427173454d61354732674644525372070401000000046e616d650a00000022566172696f75732053656375726974792054657374696e67205363656e6172696f730704010000000673796d626f6c0a00000005765365635407040100000008746f6b656e5f69640a00000004393030300704010000000a746f6b656e5f747970650a00000003464132`
  - Amount: `21`
  - Receiver: `KT1RZKsU9CD4siruhkxwzuwshwmmdXeM2ia4`
Successfully transfered, tx hash: `onvymB4uVYxmhL31jMu3GhZ2sGbLPMGCdMbRXfF8E7hmRXbwH54`


In [48]:
result = get_tickets_count(ETHERLINK_ROLLUP_NODE_URL, ticket, token_proxy_tester.address)
result

1000

### The following checked with 1000 > amount > 990 and it fails, but 990 works:

In [50]:
result = withdraw.callback(
    erc20_proxy_address=token_proxy_tester.address,
    tezos_side_router_address=ticketer.address,
    amount=990,
    # TODO: consider changing this to ticketer_address_bytes_hex ?
    ticketer_address_bytes=make_address_bytes(ticketer.address),
    # TODO: consider changing this to ticket_content_bytes_hex ?
    ticket_content_bytes=ticketer.read_content().to_bytes_hex(),
    receiver_address=get_address(tezos_account),
    withdraw_precompile=FA_WITHDRAWAL_PRECOMPILE,
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL,
)

Making FA withdrawal, ERC20 token: `0x11e1D330879bC3cc54c0F698Dd080F326C6BD5c4`:
  - Executor: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://etherlink.dipdup.net`
  - Withdrawal params:
      * Ticket owner: `0x11e1D330879bC3cc54c0F698Dd080F326C6BD5c4`
      * Receiver: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
      * Router: `KT1TQiSYR6VhFHso5xkEYR3Fgg5Vsz8iRaK7`
      * Routing info: `0x0000d1b03118754a8e193b3e5d3a7cded976f4fd425601ce839f3e5187c0abec444760a58e950b675a334100`
      * Amount: `990`
      * Ticketer address bytes: `0x01ce839f3e5187c0abec444760a58e950b675a334100`
      * Content bytes: `0x0707000005090a000000c00502000000ba07040100000010636f6e74726163745f616464726573730a000000244b54314b6639633575664c574538394e347a425775427173454d61354732674644525372070401000000046e616d650a00000022566172696f75732053656375726974792054657374696e67205363656e6172696f730704010000000673796d626f6c0a00000005765365635407040100000008746f6b656e5f69640a0000000439303030070

In [51]:
result

'0xe563c54dab81fb26a8df3694316986546b484dcbbee2128857fd783f0aebf217'

In [52]:
get_tickets_count(ETHERLINK_ROLLUP_NODE_URL, ticket, token_proxy_tester.address)

0

### OK, lets deposit when contract tries to make withdrawal when there are no tickets, but deposit still should succeed because even if it failed the receiver should get tickets and receiver is the TokenProxyTester:

In [53]:
ticket = ticketer.read_ticket(tezos_account)
ticket, _ = ticket.split(15)

opg = transfer_tickets(
    ticket=ticket,
    sender=tezos_account,
    receiver=ticket_router_tester,
)

Transfering tickets:
  - Sender: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Ticketer address: `KT1TQiSYR6VhFHso5xkEYR3Fgg5Vsz8iRaK7`
  - Ticket content: `0x0707000005090a000000c00502000000ba07040100000010636f6e74726163745f616464726573730a000000244b54314b6639633575664c574538394e347a425775427173454d61354732674644525372070401000000046e616d650a00000022566172696f75732053656375726974792054657374696e67205363656e6172696f730704010000000673796d626f6c0a00000005765365635407040100000008746f6b656e5f69640a00000004393030300704010000000a746f6b656e5f747970650a00000003464132`
  - Amount: `15`
  - Receiver: `KT1RZKsU9CD4siruhkxwzuwshwmmdXeM2ia4`
Successfully transfered, tx hash: `oosGgfeP3nXJRgeKe2vpwmgNze4oqp4Kj3WpDWgTzUpp2Xhy522`


In [61]:
get_tickets_count(ETHERLINK_ROLLUP_NODE_URL, ticket, token_proxy_tester.address)

15

### Deposit one more time:

In [165]:
ticket = ticketer.read_ticket(tezos_account)
ticket, _ = ticket.split(1)

opg = transfer_tickets(
    ticket=ticket,
    sender=tezos_account,
    receiver=ticket_router_tester,
)

Transfering tickets:
  - Sender: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Ticketer address: `KT1TQiSYR6VhFHso5xkEYR3Fgg5Vsz8iRaK7`
  - Ticket content: `0x0707000005090a000000c00502000000ba07040100000010636f6e74726163745f616464726573730a000000244b54314b6639633575664c574538394e347a425775427173454d61354732674644525372070401000000046e616d650a00000022566172696f75732053656375726974792054657374696e67205363656e6172696f730704010000000673796d626f6c0a00000005765365635407040100000008746f6b656e5f69640a00000004393030300704010000000a746f6b656e5f747970650a00000003464132`
  - Amount: `1`
  - Receiver: `KT1RZKsU9CD4siruhkxwzuwshwmmdXeM2ia4`
Successfully transfered, tx hash: `ooSwjstkK2xcWKrJJNVvbc2hihbGf2WrrwpCkLhADCRhaPNRbts`


In [166]:
get_tickets_count(ETHERLINK_ROLLUP_NODE_URL, ticket, token_proxy_tester.address)

6

### User withdrawals 6 tickets, this shouldn't be enough for internal withdrawal of 10 tickets:

In [ ]:
result = withdraw.callback(
    erc20_proxy_address=token_proxy_tester.address,
    tezos_side_router_address=ticketer.address,
    amount=6,
    # TODO: consider changing this to ticketer_address_bytes_hex ?
    ticketer_address_bytes=make_address_bytes(ticketer.address),
    # TODO: consider changing this to ticket_content_bytes_hex ?
    ticket_content_bytes=ticketer.read_content().to_bytes_hex(),
    receiver_address=get_address(tezos_account),
    withdraw_precompile=FA_WITHDRAWAL_PRECOMPILE,
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL,
)

## ?
- Scenario B: `TokenProxyTester` withdraws tickets twice during withdrawal, but it has not enough tickers
- setup:
    - tester originated with callsCount=0
    - deposit 10 tickets to the proxy from L1 via kernel
    - tester resetup to have callsCount=1, amount=10
- test:
    - user calls `fa_withdrawal_precompile` to withdraw 1 ticket from proxy
        * tester tries to withdraw 10 tickets more in the internal call
    - check that transaction failed
    - check that `fa_withdrawal_precompile` not broken after this failure

## ?
- Scenario C: `TokenProxyTester` withdraws tickets during deposit
- setup:
    - tester originated with callsCount=1, amount=11
- test:
    - deposit 10 tickets to the proxy from L1 via kernel
        * tester tries to withdraw 11 tickets and fails

## ?
- Scenario D: with successful withdrawal during deposit (withdraw less than deposit)
- TODO: consider making abstract test for deposit/withdrawal with different setups?
    - some kind of fuzz testing? parametrize all of the scenarios into one and check them all? and try to find more?

### Outbox Enumerator

In [65]:
from scripts.helpers.etherlink.contract import (
    EvmContractHelper,
    make_filename,
)

class OutboxEnumeratorHelper(EvmContractHelper): ...


outbox_enumerator = OutboxEnumeratorHelper.originate_from_file(
    web3=web3,
    account=etherlink_account,
    filename=make_filename('OutboxEnumerator'),
    constructor_args=(
        XTZ_WITHDRAWAL_PRECOMPILE,
        450,
    )
)
outbox_enumerator.address

'0x4da03736c18D4DDbCe44bED5039bE08B86360834'

In [66]:
from scripts.tezos import xtz_deposit
xtz_deposit.callback(
    xtz_ticket_helper=XTZ_TICKET_HELPER,
    tezos_private_key=TEZOS_PRIVATE_KEY,
    tezos_rpc_url=TEZOS_RPC_URL,
    amount=1_000_000,
    receiver_address=outbox_enumerator.address,
    smart_rollup_address=SMART_ROLLUP_ADDRESS,
)

Making XTZ deposit using Helper `KT1MJxf4KVN3sosR99VRG7WBbWTJtAyWUJt9`:
  - Executor: `tz1ekkzEN2LB1cpf7dCaonKt6x9KVd9YVydc`
  - Tezos RPC node: `https://rpc.tzkt.io/parisnet/`
  - XTZ deposit params:
      * Smart Rollup address: `sr1G9N4WzRF79GeGKVd1vUDUg2NSrFGNGAfA`
      * Receiver address: `0x4da03736c18D4DDbCe44bED5039bE08B86360834`
      * Amount: `1000000`
Successfully executed XTZ deposit, tx hash: `onqXvAsxKaNzPMqUr5Wom9k4h6KoXMRDWt9XjJEraeVPdvqEJDe`


'onqXvAsxKaNzPMqUr5Wom9k4h6KoXMRDWt9XjJEraeVPdvqEJDe'

In [67]:
from scripts.etherlink import xtz_withdraw

xtz_withdraw.callback(
    amount=10**12,
    # xtz_withdraw_precompile=XTZ_WITHDRAWAL_PRECOMPILE,
    xtz_withdraw_precompile=outbox_enumerator.address,
    receiver_address='tz1burnburnburnburnburnburnburjAYjjX',
    etherlink_private_key=ETHERLINK_PRIVATE_KEY,
    etherlink_rpc_url=ETHERLINK_RPC_URL
)

Making XTZ withdrawal from `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`:
  - Executor: `0x7e6f6CCFe485a087F0F819eaBfDBfb1a49b97677`
  - Etherlink RPC node: `https://etherlink.dipdup.net`
  - Withdrawal params:
      * Receiver: `tz1burnburnburnburnburnburnburjAYjjX`
      * Amount (wei):: `1000000000000`
      * Amount (mutez):: `1`
Successfully initiated XTZ withdrawal, tx hash: `0xd42be4ce04b3a49502478c37ef4992e37aa07611b1fdd1796147786549b90262`


'0xd42be4ce04b3a49502478c37ef4992e37aa07611b1fdd1796147786549b90262'